In [46]:
import numpy as np
import cv2
import time
import imutils

# test with dot image
image = cv2.imread("dots.jpg")

# config the color range
lower_red_bgr=np.array([0, 0, 180])
upper_red_bgr=np.array([60, 60, 255])

lower_red_hsv_a = np.array([170,125,0])
upper_red_hsv_a = np.array([180,255,255])

lower_red_hsv_b = np.array([0,125,0])
upper_red_hsv_b = np.array([10,255,255])

# start the capture
cap = cv2.VideoCapture(0)
time.sleep(1.0)
if not cap.isOpened():
    print("Cam open failed")
else:
#     cap.set(cv2.CAP_PROP_FRAME_WIDTH,1920); # 1080p
#     cap.set(cv2.CAP_PROP_FRAME_HEIGHT,1080); # 1080p
    
#     cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280); # 720p
#     cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720); # 720p
    
    # Check the resolution
    frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                  int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    print('res'),print(frame_size)
    
    ret, frame = cap.read()
    cv2.imshow('Preview',frame)

    # print(cap.get(cv2.CAP_PROP_FPS))
    # cap.set(cv2.CAP_PROP_FPS, 30)
    print(cap.get(cv2.CAP_PROP_FPS))

# main loop
old = time.time()
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
    hsv=cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)
#     mask_bgr=cv2.inRange(hsv,lower_red_bgr,upper_red_bgr)
    mask_bgr_a = cv2.inRange(hsv,lower_red_hsv_a,upper_red_hsv_a)
    mask_bgr_b = cv2.inRange(hsv,lower_red_hsv_b,upper_red_hsv_b)
    mask_bgr = mask_bgr_a + mask_bgr_b
    #remove noise
    mask_bgr = cv2.erode(mask_bgr, None, iterations=2)
    mask_bgr = cv2.dilate(mask_bgr, None, iterations=2)
    
    # find the objects
    cnts = cv2.findContours(mask_bgr.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cnts
    cnts = imutils.grab_contours(cnts)
#     cnts = sorted(cnts, key=lambda x: cv2.contourArea(x))
#     cnts = sorted(cnts, key=cv2.contourArea)
    center = list()

    n = min(len(cnts),4)
    for i in range(n):
        # find the 4 largest contours in the mask, then use
        # it to compute the centroid
        c = cnts[i]
        M = cv2.moments(c)
        center.append((int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])))
        # only proceed if the radius meets a minimum size
        if radius > 5:
            # draw the Contours and centroid on the frame
            cv2.drawContours(frame, contours, i,  (255, 0, 0), 1)
            cv2.circle(frame, center[i], 1, (0, 255, 0), -1)
    
    # Trigonometry comes here
    # TODO
    # calculate the distances and the angle
    print(center)
    
    
    # Communication comes here
    # TODO
    # send the information to the main script
    # https://medium.com/python-pandemonium/python-socket-communication-e10b39225a4c
    
    
    # Display the number of the center
    for i in range(len(center)):
        cv2.putText(frame, str(i), center[i], cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 255, 0))
    # Display the frames per second
    now = time.time()
    fps = 1/(now - old)
    fps_text = "FPS : %0.0f" % fps
    old = now
    cv2.putText(frame, fps_text, (0, 10), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 255, 0))
    
    # Display the resulting frame
    cv2.imshow('Preview',frame)
    
    # exit on ESC
    key = cv2.waitKey(1)
    if key == 27:
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

res
(640, 480)
30.0
[(157, 335)]
[(158, 335)]
[(158, 334)]
[(158, 334)]
[(158, 335)]
[(158, 334)]
[(149, 398), (140, 397), (158, 334)]
[(158, 334)]
[(158, 334)]
[(158, 334)]
[(158, 334)]
[(149, 398), (158, 333)]
[(158, 334)]
[(158, 333)]
[(158, 333)]
[(158, 333)]
[(158, 333)]
[(158, 333)]
[(158, 333)]
[(158, 334)]
[(158, 333)]
[(147, 397), (158, 333)]
[(158, 334)]
[(158, 333)]
[(158, 333)]
[(149, 397), (158, 334)]
[(158, 334)]
[(158, 334)]
[(158, 334)]
[(158, 333)]
[(158, 334)]
[(158, 333)]
[(158, 333)]
[(148, 398), (158, 334)]
[(158, 333)]
[(158, 334)]
[(158, 333)]
[(158, 333)]
[(148, 396), (158, 335)]
[(158, 333)]
[(158, 334)]
[(158, 333)]
[(158, 334)]
[(158, 334)]
[(159, 333)]
[(159, 333)]
[(158, 334)]
[(158, 334)]
[(158, 334)]
[(158, 333)]
[(158, 335)]
[(158, 333)]
[(158, 334)]
[(147, 397), (159, 333)]
[(159, 333)]
[(159, 334)]
[(396, 362), (159, 333)]
[(159, 334)]
[(159, 334)]
[(152, 397), (159, 334)]
[(159, 334)]
[(159, 334)]
[(142, 401), (159, 334)]
[(159, 334)]
[(159, 334)]
[(1

[(288, 419), (252, 418), (212, 415), (121, 406)]
[(261, 420), (237, 419), (212, 416), (118, 405)]
[(251, 422), (260, 421), (214, 418), (205, 401)]
[(249, 422), (211, 418), (206, 402), (120, 408)]
[(230, 421), (211, 419), (207, 403), (119, 408)]
[(102, 420), (208, 420), (133, 419), (83, 420)]
[(208, 420), (209, 406), (119, 409), (258, 310)]
[(208, 420), (209, 405), (117, 406), (232, 264)]
[(207, 421), (117, 412), (118, 393), (232, 262)]
[(206, 422), (117, 410), (233, 263), (365, 209)]
[(117, 408), (233, 264), (366, 210), (239, 120)]
[(117, 408), (233, 264), (366, 210), (239, 120)]
[(117, 409), (262, 312), (234, 266), (366, 212)]
[(117, 417), (234, 270), (366, 214), (238, 123)]
[(271, 313), (233, 271), (366, 216), (237, 125)]
[(117, 420), (234, 276), (366, 218), (321, 176)]
[(233, 278), (366, 221), (235, 131)]
[(235, 286), (321, 185), (367, 227), (235, 138)]
[(265, 334), (237, 288), (323, 187), (369, 229)]
[(117, 422), (267, 338), (240, 290), (372, 230)]
[(116, 387), (276, 336), (115, 33

[(150, 348), (115, 302), (487, 214), (248, 240)]
[(115, 303), (487, 214), (247, 240), (111, 151)]
[(150, 349), (116, 303), (486, 214), (248, 241)]
[(117, 306), (487, 215), (248, 241), (111, 152)]
[(151, 350), (117, 305), (248, 243), (112, 154)]
[(151, 350), (117, 305), (248, 243), (112, 154)]
[(118, 307), (249, 244), (112, 155), (211, 66)]
[(119, 309), (250, 246), (113, 157), (212, 70)]
[(157, 354), (120, 311), (250, 248), (115, 159)]
[(122, 314), (252, 250), (116, 161), (214, 74)]
[(123, 316), (253, 253), (117, 164), (215, 77)]
[(129, 325), (258, 262), (123, 173), (220, 86)]
[(134, 331), (263, 270), (129, 180), (225, 95)]
[(142, 341), (270, 280), (138, 192), (233, 108)]
[(155, 353), (279, 295), (150, 207), (243, 126)]
[(171, 367), (289, 313), (166, 225), (256, 148)]
[(188, 383), (302, 333), (184, 247), (638, 207)]
[(188, 383), (302, 333), (184, 247), (638, 207)]
[(205, 397), (315, 353), (202, 272), (624, 168)]
[(220, 407), (330, 371), (221, 296), (156, 204)]
[(236, 416), (117, 392), (